In [2]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation
from tensorflow.keras.optimizers import SGD, Adam

import random 
from collections import deque

import cv2
import os
from tqdm import tqdm
import json

In [3]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [35]:
def softmax(x) :
    e_x= np.exp(x - np.max(x))
    return e_x / e_x.sum()

class DQNAgent :
    def __init__ (
        self, 
        env,
        num_actions,
        lr,
        discount,
        batch_size = 32,
        mem_size = 100000,
        prefetch_size = 10000,
        eps = 1.,
        eps_decay = 0.99,
        eps_min =0.02    
    ) :
        self.env = env
        self.n_actions = num_actions
        state = self.env.reset()
        self.state_shape = self.preprocess(state).shape
        print("state_shape : ",self.state_shape)
        self.discount = discount
        
        #eps
        self.eps = eps
        self.eps_decay_rate = eps_decay
        self.eps_min = eps_min
        
        #training parameters
        self.lr = lr
        self.batch_size = batch_size
        self.prefetch_size = prefetch_size
        
        #memory
        self.mem_size = mem_size
        self.mem = deque(maxlen = self.mem_size)
        
        #network
        self.model = self._build_model()
        
    def _build_model(self):
        '''
        Build the network using keras
        '''
        model = Sequential()
        model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same',input_shape=self.state_shape[1:]))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dense(self.n_actions))
        
        adam = Adam(lr=self.lr)
        model.compile(loss='mse',optimizer=adam)

        return model
        
    def preprocess (self, state) : #rehsape state to fit our model
        return np.concatenate(state, axis = -1)[np.newaxis,:,:,:]
    
    
    def act(self, state, random_en = False): #return best action based on current Qs
        if len(state) == 4 :
            state = self.preprocess(state)
                
        if random_en :
            eps = self.eps
        else :
            eps = 0
            
        if random.random() <= eps :
            return random.randrange(self.n_actions) + 2
        else :
            Qs = self.model.predict(state)[0]
            return np.argmax(softmax(Qs))+2
        
    def remember(self, state, action, reward, next_state, done): # save to experience replay buffer
        self.mem.append((state,action, reward, next_state, done))
        
    def replay(self) : #Main experience replay routine
        minibatch = random.sample(self.mem, self.batch_size)
        
        states = []
        targets = []
        
        for state, action, reward, next_state, done in minibatch :
            Q = reward
            if not done :
                next_Qs = self.model.predict(next_state)
                Q = reward + self.discount*np.amax(next_Qs) 
             
            target_Qs = self.model.predict(state)
            target_Qs[0][action-2] = Q
            states.append(state[0])
            targets.append(target_Qs[0])
            
        self.model.fit(np.stack(states,axis = 0), np.stack(targets,axis = 0), epochs=1, verbose = 0)
        
    
    def train(self, timesteps, weight_file = None, save_steps = 10000) : #
        '''
        Train agent
        :param timesteps: timesteps to train agent with
        :param weights_file: file path to save trained weights
        :param save_step: when to save model
        '''
        
        state = self.env.reset()
        state = self.preprocess(state)
        
        for t in tqdm(range(1, timesteps+1)) :
            action = self.act(state, random_en = True)
            
            if t %1000 == 0:
                self.eps = max(self.eps_min, self.eps_decay_rate*self.eps)
                
            next_state, reward, done, _ = self.env.step(action)
            next_state = self.preprocess(next_state)
            self.remember(state, action, reward, next_state, done)
            state = next_state
            
            if len(self.mem) > self.prefetch_size :
                self.replay()
                
            if done :
                state = self.env.reset()
                state = self.preprocess(state)
            
            if t % save_steps == 0 and weight_file is not None :
                self.save_model(weight_file)
                
    def save_model(self,weights_files) :
        self.model.save_weights(weights_files+'.h5', overwrite = True)
        with open(weights_files+".json", 'w') as outfile :
            self.dump(self.model.to_json, outfile)
            
        print("model saved....")
        
    def load_model(self,weights_files) :
        self.model.load_weights(weights_files+'.h5')
            
        print("model loaded....")
    

In [5]:
#wrappers

import numpy as np
import os
os.environ.setdefault('PATH', '')
from collections import deque
import gym
from gym import spaces
import cv2
cv2.ocl.setUseOpenCL(False)

class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)

class ClippedRewardsWrapper(gym.RewardWrapper):
    def reward(self, reward):
        """Change all the positive rewards to 1, negative to -1 and keep zero."""
        return np.sign(reward)

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = width
        self.height = height
        self.grayscale = grayscale
        if self.grayscale:
            self.observation_space = spaces.Box(low=0, high=255,
                shape=(self.height, self.width, 1), dtype=np.uint8)
        else:
            self.observation_space = spaces.Box(low=0, high=255,
                shape=(self.height, self.width, 3), dtype=np.uint8)

    def observation(self, frame):
        if self.grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        if self.grayscale:
            frame = np.expand_dims(frame, -1)
        return frame

class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[:-1] + (shp[-1] * k,)), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return list(self.frames)

class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=-1)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

def make_atari(env_id, timelimit=True):
    # XXX(john): remove timelimit argument after gym is upgraded to allow double wrapping
    env = gym.make(env_id)
    if not timelimit:
        env = env.env
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env

In [6]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


In [7]:
#main.py
import gym
#from DQN import DQNAgent
import numpy as np
import random
import time
#from atari_wrappers import *

In [ ]:
#main.py
train_steps = 400000
lr = 0.0001
gamma = 0.99 #discount
num_actions = 2
#import DQNAgent

trials = 10 #avaluation trials

def wrap_dqn(env):
    """Apply a common set of wrappers for Atari games."""
    assert 'NoFrameskip' in env.spec.id
    env = EpisodicLifeEnv(env)
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = ProcessFrame84(env)
    env = FrameStack(env, 4)
    env = ClippedRewardsWrapper(env)
    return env

env = wrap_dqn(gym.make('PongNoFrameskip-v4'))

agent = DQNAgent(env = env, lr = lr, discount = gamma, num_actions = num_actions)

agent.train(timesteps = 400000)#, weight_file = "snaps/model")

#evaluation
success = 0
for tr in (trials):
    state = env.reset()
    t = 0
    acc_r  = 0
    
    while True :
        env.render()
        action = agent.act(state)
    
        state, reward, done, _= env.step(action)
        acc_R += reward
        t+=1
        
        if done :
            print("trials {} finished after {}".format(tr,t))
            
            if acc_r>0 :
                success +=1
            break
            
print("success %d/%d"%(success,trials))

env.close()

In [ ]:
pip install gym[atari]

In [ ]:
pip install git+https://github.com/Kojoley/atari-py.git

In [ ]:
pip install setuptools gitpython

In [ ]:
sudo add-apt-repository ppa:git-core/ppa
sudo apt update
sudo apt install git

In [ ]:
pip install --no-index -f https://github.com/Kojoley/atari-py/releases atari_py

In [ ]:
#,accept-rom-license]==0.21